In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adityajn105","key":"571939f74e6a6e23ec01a7d769da7958"}'}

In [0]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/
!kaggle competitions download -c recognizing-faces-in-the-wild
!mkdir train test
!unzip -q train.zip -d train
!unzip -q test.zip -d test
!rm train.zip test.zip

  0% 0.00/77.6k [00:00<?, ?B/s]
100% 77.6k/77.6k [00:00<00:00, 31.0MB/s]
 50% 17.0M/34.1M [00:00<00:00, 21.8MB/s]
100% 34.1M/34.1M [00:00<00:00, 43.8MB/s]
 92% 63.0M/68.6M [00:01<00:00, 19.9MB/s]
100% 68.6M/68.6M [00:01<00:00, 38.4MB/s]
  0% 0.00/156k [00:00<?, ?B/s]
100% 156k/156k [00:00<00:00, 162MB/s]


In [0]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

In [1]:
import gc
import glob
from keras.preprocessing.image import img_to_array, load_img, array_to_img
from keras_vggface.utils import preprocess_input

IMG_DIM = (224,224,3)

def read_img(path):
  return preprocess_input( img_to_array( load_img(path, target_size=IMG_DIM) ),version=2)

Using TensorFlow backend.


In [0]:
allPhotos = dict()
for family in glob.glob("train/*"):
  for mem in glob.glob(family+'/*'):
    allPhotos[mem] = []
    for photo in glob.glob(mem+'/*'):
      allPhotos[mem].append(read_img(photo))

In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('train_relationships.csv')
data.p1 = data.p1.apply( lambda x: 'train/'+x )
data.p2 = data.p2.apply( lambda x: 'train/'+x )
data['related'] = 1 
print(data.shape)
data.head()

(3598, 3)


,p1,p2,related
0,train/F0002/MID1,train/F0002/MID3,1
1,train/F0002/MID2,train/F0002/MID3,1
2,train/F0005/MID1,train/F0005/MID2,1
3,train/F0005/MID3,train/F0005/MID2,1
4,train/F0009/MID1,train/F0009/MID4,1


In [0]:
from itertools import combinations
families = glob.glob('train/*')
p1 = []; p2 = []
for f1,f2 in combinations(families,2) :
  for _p1 in glob.glob( '{}/*'.format(f1) ):
    for _p2 in glob.glob( '{}/*'.format(f2) ):
      p1.append( _p1 ); p2.append( _p2 );
temp = pd.DataFrame({'p1':p1,'p2':p2,'related':np.zeros( (len(p1),) ,dtype=np.int32)})

In [0]:
temp = temp.sample(n=data.shape[0]*2)
data = data.append(temp).sample(frac=1.).reset_index().drop(['index'],axis=1)
print(data.shape)
data.head()

(10794, 3)


,p1,p2,related
0,train/F0030/MID3,train/F0438/MID2,0
1,train/F0130/MID3,train/F0130/MID1,1
2,train/F0812/MID1,train/F0831/MID7,0
3,train/F1000/MID7,train/F0756/MID3,0
4,train/F0203/MID3,train/F0203/MID5,1


In [0]:
data['p1l'] = data.p1.apply( lambda x: len(allPhotos[x]) if x in allPhotos.keys() else 0  )
data['p2l'] = data.p2.apply( lambda x: len(allPhotos[x]) if x in allPhotos.keys() else 0 )
data = data[ (data.p1l!=0) & (data.p2l!=0) ]
data.head()

,p1,p2,related,p1l,p2l
0,train/F0030/MID3,train/F0438/MID2,0,8,14
1,train/F0130/MID3,train/F0130/MID1,1,6,11
2,train/F0812/MID1,train/F0831/MID7,0,13,1
3,train/F1000/MID7,train/F0756/MID3,0,2,4
4,train/F0203/MID3,train/F0203/MID5,1,1,2


In [0]:
del temp, p1, p2; gc.collect()

29

In [0]:
from sklearn.model_selection import train_test_split
train, val, Y_train, Y_val = train_test_split( data[['p1','p2']], data.related, test_size=0.3, stratify = data.related,  )
train.shape,val.shape,Y_train.shape,Y_val.shape

((7382, 2), (3164, 2), (7382,), (3164,))

In [0]:
train.reset_index(inplace=True)
val.reset_index(inplace=True)
Y_train = Y_train.reset_index()
Y_val = Y_val.reset_index()

In [0]:
def getImages(p1,p2):
    _p1 = allPhotos[p1];_p2 = allPhotos[p2]
    return _p1[np.random.randint(len(_p1))], _p2[np.random.randint(len(_p2))]

def getMiniBatch(batch_size=32, path = train, y = Y_train):
  p1 = []; p2 = []; Y = []
  indexes = np.random.choice( path[y==1].index, size=int(batch_size/2), replace=False )
  for index in indexes:
    _p1, _p2 = getImages( path.iloc[index].p1 , path.iloc[index].p2 )
    p1.append(_p1);p2.append(_p2);Y.append(1)
  indexes = np.random.choice( path[y==0].index, size=int(batch_size/2), replace=False )
  for index in indexes:
    _p1, _p2 = getImages( path.iloc[index].p1 , path.iloc[index].p2 )
    p1.append(_p1);p2.append(_p2);Y.append(0)
  return [np.array(p1),np.array(p2)], np.array(Y)

In [0]:
from sklearn.metrics import roc_auc_score
def test_oneshot(model,N,verbose=0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    inputs, targets = getMiniBatch(N, path = val, y = Y_val)
    probs = model.predict(inputs)
    auc = roc_auc_score( targets, probs, )
    if verbose:
        print("Got an AUC of {:.4f} in {} way one-shot learning.".format(auc,N))
    return auc

In [3]:
from keras_vggface.vggface import VGGFace
vggface = VGGFace(model='resnet50', include_top=False, input_shape=IMG_DIM)
vggface.trainable=False
for layer in vggface.layers[:-3]:
  layer.trainable=False

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
def initialize_bias(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)
  
def initialize_weights(shape, name=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [5]:
from keras.layers import Input, Dense, Flatten
from keras.layers import Lambda, Subtract
from keras.models import Model
from keras.regularizers import l2
from keras import backend as K
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy
import tensorflow as tf
from sklearn.metrics import roc_auc_score

def auc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

left_input = Input(IMG_DIM)
right_input = Input(IMG_DIM)


x1 = vggface(left_input)
x2 = vggface(right_input)

flatten = Flatten()
x1 = flatten(x1)
x2 = flatten(x2)

fc = Dense(4096,activation='relu',kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias)

encoded_l = fc(x1)
encoded_r = fc(x2)

L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
L1_distance = L1_layer([encoded_l, encoded_r])

prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)

siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

optimizer = Adam(1e-3)

siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=['accuracy',auc])

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    


In [0]:
class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self): # Py3
        with self.lock:
            return next(self.it)
          
def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g
  
  
@threadsafe_generator
def Generator(batch_size, path,y):
  while True:
    yield getMiniBatch(batch_size=batch_size,path=path, y=y)

In [24]:
from keras.callbacks import ReduceLROnPlateau
import threading

reducelr = ReduceLROnPlateau(monitor='val_loss',mode='max',patience=20,min_lr=1e-5,verbose=1)

callbacks_list = [reducelr]


train_gen = Generator(batch_size=32,path=train,y=Y_train)
val_gen = Generator(batch_size=32,path=val,y=Y_val)

siamese_net.fit_generator( train_gen, samples_per_epoch=100, epochs=1000, 
                          validation_data=val_gen, validation_steps=50, 
                          verbose=1,callbacks=callbacks_list, workers=4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., epochs=1000, validation_data=<__main__...., validation_steps=50, verbose=1, callbacks=[<keras.ca..., workers=4, steps_per_epoch=100)`
  


Epoch 1/1000
100/100 [==============================] - 73s 726ms/step - loss: 3.0911 - acc: 0.5056 - auc: 0.5066 - val_loss: 2.4185 - val_acc: 0.4931 - val_auc: 0.4960
Epoch 2/1000
100/100 [==============================] - 70s 701ms/step - loss: 2.0042 - acc: 0.4984 - auc: 0.4962 - val_loss: 1.8431 - val_acc: 0.5062 - val_auc: 0.4966
Epoch 3/1000
100/100 [==============================] - 70s 702ms/step - loss: 1.7236 - acc: 0.5031 - auc: 0.4980 - val_loss: 1.6931 - val_acc: 0.4963 - val_auc: 0.4979
Epoch 4/1000
100/100 [==============================] - 70s 703ms/step - loss: 1.6707 - acc: 0.4937 - auc: 0.4983 - val_loss: 1.6087 - val_acc: 0.4994 - val_auc: 0.4984
Epoch 5/1000
100/100 [==============================] - 70s 702ms/step - loss: 1.6297 - acc: 0.4894 - auc: 0.4979 - val_loss: 1.6602 - val_acc: 0.4806 - val_auc: 0.4980
Epoch 6/1000
100/100 [==============================] - 70s 701ms/step - loss: 1.6289 - acc: 0.5006 - auc: 0.4976 - val_loss: 1.6099 - val_acc: 0.4906 - va

KeyboardInterrupt: ignored

In [0]:
siamese_net.save_weights('siamesenet_weights.h5')

In [0]:
siamese_net.load_weights('siamesenet_weights.h5')

In [7]:
del allPhotos; gc.collect()

0

In [19]:
submission = pd.read_csv('sample_submission.csv')
submission['p1'] = submission.img_pair.apply( lambda x: 'test/'+x.split('-')[0] )
submission['p2'] = submission.img_pair.apply( lambda x: 'test/'+x.split('-')[1] )
print(submission.shape)
submission.head()

(5310, 4)


,img_pair,is_related,p1,p2
0,face05508.jpg-face01210.jpg,0,test/face05508.jpg,test/face01210.jpg
1,face05750.jpg-face00898.jpg,0,test/face05750.jpg,test/face00898.jpg
2,face05820.jpg-face03938.jpg,0,test/face05820.jpg,test/face03938.jpg
3,face02104.jpg-face01172.jpg,0,test/face02104.jpg,test/face01172.jpg
4,face02428.jpg-face05611.jpg,0,test/face02428.jpg,test/face05611.jpg


In [20]:
from tqdm import tqdm

probs = []
for i,j in tqdm([ (0,1000),(1000,2000),(2000,3000),(3000,4000),(4000,5310) ]):
  imgs1 = np.array( [ read_img(photo) for photo in submission.p1.values[i:j] ] )
  imgs2 = np.array( [ read_img(photo) for photo in submission.p2.values[i:j] ] )
  prob =  siamese_net.predict( [ imgs1, imgs2 ] )
  probs.append(np.squeeze(prob))
  del imgs1,imgs2; gc.collect()

100%|██████████| 5/5 [01:53<00:00, 23.57s/it]


In [0]:
submission.is_related = np.concatenate(probs)
submission.drop( ['p1','p2'],axis=1,inplace=True )
submission.head()

In [0]:
submission.to_csv('submission.csv',index=False)

In [17]:
!kaggle competitions submit -c recognizing-faces-in-the-wild -f submission.csv -m "1st prob vgg-face basic"

100% 199k/199k [00:05<00:00, 35.0kB/s]
Successfully submitted to Northeastern SMILE Lab - Recognizing Faces in the Wild

In [0]:
import pandas as pd
layers = [(layer, layer.name, layer.trainable) for layer in vggface.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at...,input_3,False
1,<keras.layers.convolutional.Conv2D object at 0...,conv1_1,False
2,<keras.layers.convolutional.Conv2D object at 0...,conv1_2,False
3,<keras.layers.pooling.MaxPooling2D object at 0...,pool1,False
4,<keras.layers.convolutional.Conv2D object at 0...,conv2_1,False
5,<keras.layers.convolutional.Conv2D object at 0...,conv2_2,False
6,<keras.layers.pooling.MaxPooling2D object at 0...,pool2,False
7,<keras.layers.convolutional.Conv2D object at 0...,conv3_1,False
8,<keras.layers.convolutional.Conv2D object at 0...,conv3_2,False
9,<keras.layers.convolutional.Conv2D object at 0...,conv3_3,False
